## Setup

### Import libraries

In [1]:
from benchmarks.experiments.wrapper_overhead.common.deployment_utility import WrapperOverheadDeploymentUtility
from benchmarks.experiments.wrapper_overhead.common.extended_aws_remote_client import ExtendedAWSRemoteClient

### Setup Common Resources

In [5]:
region = 'us-east-2'
client: ExtendedAWSRemoteClient = ExtendedAWSRemoteClient(region)
deployment_utility: WrapperOverheadDeploymentUtility = WrapperOverheadDeploymentUtility(region)

## Deploy Associated AWS functions

### DNA Visualization

- For MultiX, please set it up according to readme contained in the file. "./dna_visualization/external_database/multi_x"

In [ ]:
dna_vis_direct_calls_dir = "./dna_visualization/external_database/boto3_only_direct_calls" # Boto3 only direct calls
dna_vis_sns_calls_dir = "./dna_visualization/external_database/boto3_only_sns" # Boto3 only sns calls
dna_vis_aws_step_functions_dir = "./dna_visualization/external_database/aws_step_function" # AWS Step Functions

deployment_utility.deploy_experiment(dna_vis_direct_calls_dir)
deployment_utility.deploy_experiment(dna_vis_sns_calls_dir)
deployment_utility.deploy_experiment(dna_vis_aws_step_functions_dir)